In [3]:
import arviz as az
import numpy as np
import pymc as pm
from pymc.math import dot, stack, concatenate, exp, invlogit, logit

%load_ext lab_black
%load_ext watermark

# Rats

This example goes further into dealing with missing data in PyMC, including in the predictor variables.

Adapted from [Unit 8: ratsignorable1.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit8/ratsignorable1.odc), [ratsignorable2.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit8/ratsignorable2.odc), and [ratsinformative.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit8/ratsinformative.odc).

Data can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/rats.txt).

Associated lecture videos: Unit 8 lesson 2

## Problem statement

We had a previous example about [Dugongs](https://areding.github.io/6420-pymc/Unit6-dugongs.html) that dealt with missing data in the observed data (y values). This example shows how to deal with missing data in the input data (x). It's still pretty easy. You could look at it like creating another likelihood in the model, a very simple one where the observed data is x, and you use a single distribution to fill in the missing values (see ```x_imputed``` in the model below).

Original paper [here.](https://www.jstor.org/stable/pdf/2289594.pdf)

Gelfand et al 1990 consider the problem of missing data, and delete the last observation of cases 6-10, the last two from 11-20, the last 3 from 21-25 and the last 4 from 26-30.  The appropriate data file is obtained by simply replacing data values by NA (see below). The model specification is unchanged, since the distinction between observed and unobserved quantities is made in the data file and not the model specification. - bugs problem statement

This first example only 

In [3]:
# adjusting the shape of x for vectorized calculations (the BUGS example is written as a loop)
x = np.array([8.0, 15.0, 22.0, 29.0, 36.0])

In [4]:
# import y data and create mask (missing data is represented as nan in the file)
y = np.loadtxt("../data/rats.txt")
y = np.nan_to_num(y, nan=-1)  # nan to -1
y = np.ma.masked_values(y, value=-1)  # create mask

original model shapes:

shapes of 1:
tau_c
alpha_c
alpha_tau
beta_c
beta_tau

shapes of 30:
alpha
beta

shapes of 30, 5:
mu
likelihood


make a note here about broadcasting, it's the only reason this works.

https://numpy.org/doc/stable/user/basics.broadcasting.html

another note: pymc doesn't seem to like gamma(.001, .001) for tau here, maybe values getting too close to 0? getting tau > 0 warning.

In [5]:
with pm.Model() as m:
    alpha_c = pm.Normal("alpha_c", 0, tau=1e-6)
    alpha_tau = pm.Gamma("alpha_tau", 0.01, 0.01)
    beta_c = pm.Normal("beta_c", 0, tau=1e-6)
    beta_tau = pm.Gamma("beta_tau", 0.01, 0.01)

    alpha = pm.Normal(
        "alpha", alpha_c, tau=alpha_tau, shape=(30, 1)
    )  # (30, 1) for broadcasting
    beta = pm.Normal("beta", beta_c, tau=beta_tau, shape=(30, 1))
    lik_tau = pm.Gamma("lik_tau", 0.01, 0.01)
    sigma = pm.Deterministic("sigma", 1 / lik_tau**0.5)

    mu = alpha + beta * x

    pm.Normal("likelihood", mu, tau=lik_tau, observed=y)

    trace = pm.sample(
        5000,
        tune=1000,
        cores=4,
    )

/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/pymc/model.py:1365: ImputationWarning: Data in likelihood contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha_c, alpha_tau, beta_c, beta_tau, alpha, beta, lik_tau, likelihood_missing]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 197 seconds.
There were 27 divergences after tuning. Increase `target_accept` or reparameterize.
There were 6 divergences after tuning. Increase `target_accept` or reparameterize.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.


In [6]:
az.summary(
    trace,
    hdi_prob=0.95,
    var_names=["alpha_c", "alpha_tau", "beta_c", "beta_tau", "sigma"],
    kind="stats",
)

,mean,sd,hdi_2.5%,hdi_97.5%
alpha_c,101.194,2.308,96.353,105.492
alpha_tau,0.028,0.091,0.004,0.052
beta_c,6.570,0.165,6.256,6.902
beta_tau,2.862,1.216,0.957,5.233
sigma,6.008,0.663,4.752,7.290


Notes:

- can't impute data with pm.Data(mutable=True)? 

    - reading: https://github.com/pymc-devs/pymc/issues/4441 https://github.com/pymc-devs/pymc/pull/5295


## Model 2: Imputing missing x data

In [7]:
# in the second example, the x data also has a missing value.
x = np.array([8.0, 15.0, 22.0, -1, 36.0])
x = np.ma.masked_values(x, value=-1)

In [8]:
x

masked_array(data=[8.0, 15.0, 22.0, --, 36.0],
             mask=[False, False, False,  True, False],
       fill_value=-1.0)

In [9]:
with pm.Model() as m:
    alpha_c = pm.Normal("alpha_c", 0, tau=1e-6)
    alpha_tau = pm.Gamma("alpha_tau", 0.01, 0.01)
    beta_c = pm.Normal("beta_c", 0, tau=1e-6)
    beta_tau = pm.Gamma("beta_tau", 0.01, 0.01)

    alpha = pm.Normal("alpha", alpha_c, tau=alpha_tau, shape=(30, 1))
    beta = pm.Normal("beta", beta_c, tau=beta_tau, shape=(30, 1))
    lik_tau = pm.Gamma("lik_tau", 0.01, 0.01)
    sigma = pm.Deterministic("sigma", 1 / lik_tau**0.5)

    x_imputed = pm.TruncatedNormal("x_imputed", mu=20, sigma=10, lower=0, observed=x)

    mu = alpha + beta * x_imputed

    pm.Normal("likelihood", mu, tau=lik_tau, observed=y)

    trace = pm.sample(5000)

/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/pymc/model.py:1365: ImputationWarning: Data in x_imputed contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/pymc/model.py:1365: ImputationWarning: Data in likelihood contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha_c, alpha_tau, beta_c, beta_tau, alpha, beta, lik_tau, x_imputed_missing, likelihood_missing]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 218 seconds.
There were 4 divergences after tuning. Increase `target_accept` or reparameterize.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.


In [10]:
az.summary(
    trace,
    hdi_prob=0.95,
    var_names=[
        "alpha_c",
        "alpha_tau",
        "beta_c",
        "beta_tau",
        "sigma",
        "x_imputed_missing",
    ],
    kind="stats",
)

,mean,sd,hdi_2.5%,hdi_97.5%
alpha_c,101.894,2.351,97.345,106.519
alpha_tau,0.021,0.048,0.004,0.043
beta_c,6.509,0.168,6.181,6.840
beta_tau,2.976,1.258,0.993,5.416
sigma,5.963,0.653,4.825,7.332
x_imputed_missing[0],29.500,0.389,28.726,30.261


## Model 3: Non-ignorable missingness

Probability of missingness increases approx. at a rate of 1% with increasing the weight.

Something is going wrong, not sure if PyMC supports this method because of the different ways missing values are handled.

In [69]:
y = np.array([177.0, 236.0, 285.0, 350.0, -1])  # original value was 320
y = np.ma.masked_values(y_arr, value=-1)  # create masked array
# note: can access mask with y.mask, equivalent to the "miss" array from the Professor's example
miss = y.mask
x = np.array([8.0, 15.0, 22.0, 29.0, 36.0])

In [76]:
_t = 0.1
s = 1 / t  # convert BUGS dlogis tau to s for pymc
b = np.log(1.01)

with pm.Model() as m:
    a = pm.Logistic("a", mu=0, s=s)
    alpha = pm.Flat("alpha")
    beta = pm.Flat("beta")
    log_sigma = pm.Flat("log_sigma")
    sigma2 = pm.Deterministic("sigma2", exp(2 * log_sigma))
    tau = pm.Deterministic("tau", 1 / sigma2)

    p = pm.Deterministic("p", invlogit(a + b * y))
    pm.Bernoulli("missing", p=p, observed=miss)

    mu = pm.Deterministic("mu", alpha + beta * x)
    pm.Normal("likelihood", mu, tau=tau, observed=y)

    trace = pm.sample(10000, tune=1000, init="jitter+adapt_diag_grad")

/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/pymc/model.py:1427: ImputationWarning: Data in likelihood contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag_grad...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a, alpha, beta, log_sigma, likelihood_missing]


Sampling 4 chains for 1_000 tune and 10_000 draw iterations (4_000 + 40_000 draws total) took 28 seconds.
There were 1165 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.6129, but should be close to 0.8. Try to increase the number of tuning steps.
There were 60 divergences after tuning. Increase `target_accept` or reparameterize.
There were 52 divergences after tuning. Increase `target_accept` or reparameterize.
There were 4135 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.2481, but should be close to 0.8. Try to increase the number of tuning steps.


In [77]:
az.summary(trace, hdi_prob=0.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
a,-4.197,1.440,-7.118,-1.455,0.056,0.042,758.0,427.0,1.00
alpha,111.778,15.617,85.735,137.411,0.220,0.168,1743.0,3770.0,1.01
beta,8.112,0.775,6.784,9.396,0.011,0.008,1970.0,3332.0,1.01
log_sigma,1.848,0.627,0.802,3.134,0.019,0.013,944.0,917.0,1.01
likelihood_missing[0],403.723,20.237,372.409,435.638,0.234,0.170,2310.0,4585.0,1.01
sigma2,183.179,2079.204,2.708,421.643,25.665,18.149,944.0,917.0,1.01
tau,0.043,0.043,0.000,0.128,0.001,0.001,944.0,917.0,1.02
p[0],0.135,0.131,0.000,0.406,0.004,0.003,758.0,427.0,1.00
p[1],0.202,0.173,0.000,0.551,0.005,0.004,758.0,427.0,1.00
p[2],0.273,0.205,0.000,0.667,0.007,0.005,758.0,427.0,1.00


In [65]:
%watermark -n -u -v -iv -p aesara,aeppl

Python implementation: CPython
Python version       : 3.10.4
IPython version      : 8.4.0

arviz: 0.12.1
pymc : 4.0.1
numpy: 1.22.4

